<a href="https://colab.research.google.com/github/DimosthenisDimoulias/datastories_for_emme/blob/main/stance-justification-thematic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load data
from google.colab import files

# Load the dataframe
uploaded = files.upload()

df = pd.read_excel(list(uploaded.keys())[0])

# Preprocessing
df['Σχόλιο'] = df['Σχόλιο'].astype(str)
df['Σχόλιο_lower'] = df['Σχόλιο'].str.lower()

# --- 1. Stance Analysis (Sentiment/Stance) ---
# Keywords
positive_keywords = ['συμφωνώ', 'θετικό', 'ορθό', 'μπράβο', 'επιτέλους', 'αναγκαίο', 'εκσυγχρονισμός', 'ευκαιρία', 'πρόοδος', 'σωστή κατεύθυνση']
negative_keywords = ['διαφωνώ', 'όχι', 'απαράδεκτο', 'απόσυρση', 'αντισυνταγματικό', 'καταστροφή', 'ντροπή', 'αίσχος', 'ιδιωτικοποίηση', 'παραβίαση', 'κατάργηση', 'εμπορευματοποίηση', 'όχι στα ιδιωτικά']

def classify_stance(text):
    text = text.lower()
    # Check for negative first as they are likely more emphatic in this context
    if any(k in text for k in negative_keywords):
        return 'Αρνητική'
    elif any(k in text for k in positive_keywords):
        return 'Θετική'
    else:
        # Heuristic: If it contains specific questions or technical terms without emotive words, maybe Neutral/Technical
        # For simplicity, if not explicitly positive or negative, we map to "Ουδέτερη/Τεχνική"
        # However, many "Withdraw it" comments might be short.
        return 'Ουδέτερη/Τεχνική'

df['Stance'] = df['Σχόλιο'].apply(classify_stance)

# --- 2. Level of Substantiation ---
# Heuristics based on length and vocabulary
def classify_substantiation(text):
    words = text.split()
    num_words = len(words)
    text_lower = text.lower()

    high_keywords = ['άρθρο 16', 'σύνταγμα', 'στει', 'νομολογία', 'ευρωπαϊκό δίκαιο', 'παρ.', 'ν.', 'συνεπώς', 'δεδομένου', 'τεκμηριώνει', 'αντίθεση', 'αιτιολογική έκθεση']

    if num_words < 15:
        return 'Μηδενική'

    # If it has significant length AND technical/legal vocabulary -> High
    if num_words > 50 and any(k in text_lower for k in high_keywords):
        return 'Υψηλή'
    # Or just very long texts often imply effort/argumentation
    if num_words > 150:
        return 'Υψηλή'

    return 'Χαμηλή'

df['Substantiation'] = df['Σχόλιο'].apply(classify_substantiation)

# --- 3. Thematic Framing ---
# Note: A comment can belong to multiple themes
themes = {
    'Συνταγματικότητα': ['σύνταγμα', 'άρθρο 16', 'αντισυνταγματικό', 'νομιμότητα', 'παραβίαση', 'στε'],
    'Ακαδημαϊκά Κριτήρια': ['εβε', 'βάση εισαγωγής', 'ποιότητα', 'αξιολόγηση', 'υποδομές', 'πτυχίο', 'εθχααε', 'εθααε', 'πιστοποίηση', 'επαγγελματικά δικαιώματα', 'αναγνώριση'],
    'Κοινωνική Δικαιοσύνη': ['δίδακτρα', 'πλούσιοι', 'φτωχοί', 'ισότητα', 'ευκαιρίες', 'δωρεάν', 'ταξικό', 'ανισότητες', 'παιδιά'],
    'Οικονομία/Ανάπτυξη': ['brain drain', 'φυγή', 'εξωτερικό', 'κύπρος', 'συνάλλαγμα', 'οικονομία', 'αγορά εργασίας', 'ανταγωνισμός', 'επενδύσεις']
}

theme_counts = {k: 0 for k in themes}

def check_themes(text):
    found_themes = []
    text = text.lower()
    for theme, keywords in themes.items():
        if any(k in text for k in keywords):
            found_themes.append(theme)
    return found_themes

df['Themes'] = df['Σχόλιο'].apply(check_themes)

# Count themes
for theme_list in df['Themes']:
    for theme in theme_list:
        theme_counts[theme] += 1

# --- VISUALIZATIONS ---

# 1. Stance Plot
plt.figure(figsize=(8, 6))
stance_counts = df['Stance'].value_counts()
colors_stance = ['#c44e52', '#55a868', '#dd8452'] # Red, Green, Orange
if 'Αρνητική' in stance_counts.index:
    colors_stance = ['#c44e52'] # Red
    if 'Θετική' in stance_counts.index: colors_stance.append('#55a868') # Green
    if 'Ουδέτερη/Τεχνική' in stance_counts.index: colors_stance.append('#dd8452') # Orange
# Mapping explicitly to ensure colors match labels
color_map_stance = {'Αρνητική': '#c44e52', 'Θετική': '#55a868', 'Ουδέτερη/Τεχνική': '#dd8452'}
plot_colors_stance = [color_map_stance.get(x) for x in stance_counts.index]

stance_counts.plot(kind='bar', color=plot_colors_stance)
plt.title('Κατανομή Στάσης Σχολιαστών')
plt.ylabel('Πλήθος Σχολίων')
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig('stance_analysis.png')
plt.close()

# 2. Substantiation Plot
plt.figure(figsize=(8, 6))
subst_counts = df['Substantiation'].value_counts().reindex(['Μηδενική', 'Χαμηλή', 'Υψηλή'])
subst_counts.plot(kind='bar', color='#4c72b0')
plt.title('Επίπεδο Τεκμηρίωσης Σχολίων')
plt.ylabel('Πλήθος Σχολίων')
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig('substantiation_analysis.png')
plt.close()

# 3. Themes Plot
plt.figure(figsize=(10, 6))
themes_series = pd.Series(theme_counts).sort_values(ascending=False)
themes_series.plot(kind='barh', color='#8172b3')
plt.title('Κυρίαρχες Θεματικές Ενότητες')
plt.xlabel('Αριθμός Αναφορών')
plt.tight_layout()
plt.savefig('thematic_analysis.png')
plt.close()

print("Stance Counts:")
print(stance_counts)
print("\nSubstantiation Counts:")
print(subst_counts)
print("\nTheme Counts:")
print(themes_series)
print("\nSample High Substantiation Comments:")
print(df[df['Substantiation']=='Υψηλή']['Σχόλιο'].head(2).tolist())
print("\nSample Negative Comments:")
print(df[df['Stance']=='Αρνητική']['Σχόλιο'].head(2).tolist())